In [112]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds

train_dir = os.path.join('chest_xray/train/')
eval_dir = os.path.join('chest_xray/test/')

In [113]:
ACCURACY_THRESHOLD = 0.95

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs={}):
        if (logs.get('accuracy')>ACCURACY_THRESHOLD):
            print("\nReached %2.2f%% accuracy, so stopping training!!" %(ACCURACY_THRESHOLD))
            self.model.stop_training = True
callbacks = myCallback()

In [114]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array,load_img

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255,zoom_range = [0.8,1.2],horizontal_flip = True,rotation_range = 20,shear_range=0.5,fill_mode='constant')

train_generator = train_datagen.flow_from_directory(
  train_dir,
  batch_size = 12,
  target_size=(100, 100),
  class_mode='binary'
)

Found 5216 images belonging to 2 classes.


In [115]:
# All images will be rescaled by 1./255
eval_datagen = ImageDataGenerator(rescale=1./255)

eval_generator = eval_datagen.flow_from_directory(
  eval_dir,
  batch_size = 12,
  target_size=(100, 100),
  class_mode='binary'
)

Found 624 images belonging to 2 classes.


In [106]:
'''
img = load_img ('chest_xray/train/PNEUMONIA/person423_bacteria_1855.jpeg')
x = img_to_array(img)
x = x.reshape((1,)+x.shape)
i = 0
for batch in train_datagen.flow(x,batch_size=1,
                         save_to_dir='augmented',save_prefix='aug',save_format="jpeg"):
    i +=1
    if i > 20:
        break
'''

In [116]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(100, 100, 3)),
    tf.keras.layers.MaxPooling2D(3, 3),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [117]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_51 (Conv2D)          (None, 98, 98, 32)        896       
                                                                 
 max_pooling2d_27 (MaxPoolin  (None, 32, 32, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_52 (Conv2D)          (None, 30, 30, 32)        9248      
                                                                 
 max_pooling2d_28 (MaxPoolin  (None, 15, 15, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_53 (Conv2D)          (None, 13, 13, 64)        18496     
                                                                 
 conv2d_54 (Conv2D)          (None, 11, 11, 64)      

In [118]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

In [119]:

history = model.fit_generator(
      train_generator,
      steps_per_epoch=434,     
      epochs=50,
      validation_data=eval_generator,
      validation_steps= 52,
      verbose=1,
      callbacks=[callbacks])

Epoch 1/50


/var/folders/k3/gcflkw4j1lx_kn_v5p132mym0000gn/T/ipykernel_957/1182610151.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(
2022-05-12 23:43:08.162247: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


434/434 [==============================] - ETA: 0s - loss: 0.4318 - accuracy: 0.8249

2022-05-12 23:43:42.227365: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


434/434 [==============================] - 37s 82ms/step - loss: 0.4318 - accuracy: 0.8249 - val_loss: 0.4826 - val_accuracy: 0.7564
Epoch 2/50
434/434 [==============================] - 35s 81ms/step - loss: 0.2509 - accuracy: 0.9016 - val_loss: 0.2987 - val_accuracy: 0.8862
Epoch 3/50
434/434 [==============================] - 36s 82ms/step - loss: 0.2153 - accuracy: 0.9245 - val_loss: 0.2937 - val_accuracy: 0.8750
Epoch 4/50
434/434 [==============================] - 35s 80ms/step - loss: 0.1967 - accuracy: 0.9333 - val_loss: 0.2742 - val_accuracy: 0.8910
Epoch 5/50
434/434 [==============================] - 35s 80ms/step - loss: 0.1839 - accuracy: 0.9345 - val_loss: 0.3266 - val_accuracy: 0.8750
Epoch 6/50
434/434 [==============================] - 35s 81ms/step - loss: 0.1620 - accuracy: 0.9427 - val_loss: 0.4235 - val_accuracy: 0.8974
Epoch 7/50
434/434 [==============================] - 35s 80ms/step - loss: 0.1738 - accuracy: 0.9412 - val_loss: 0.2544 - val_accuracy: 0.9006
Epo

In [120]:
from keras.preprocessing import image



img =  image.load_img('norm.jpeg',target_size=(100,100))

x = image.img_to_array(img)
x = np.expand_dims(x,axis=0)


images = np.vstack([x])
classes = model.predict(images,batch_size=10)

print (classes)



[[3.686933e-22]]


2022-05-12 23:50:21.296547: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [121]:
from keras.preprocessing import image



img =  image.load_img('chest_xray/train/PNEUMONIA/person348_bacteria_1603.jpeg',target_size=(100,100))

x = image.img_to_array(img)
x = np.expand_dims(x,axis=0)


images = np.vstack([x])
classes = model.predict(images,batch_size=10)

print (classes)



[[0.96519697]]
